# Convolutional Neural Networks

In [1]:
import tensorflow as tf

ImportError: No module named tensorflow

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [3]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


## Basic Model

In [4]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [5]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

In [8]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [10]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [11]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [12]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1),
          'Avg. cost = {:,.3f}'.format(total_cost / total_batch))

Epoch: 0001 Avg. cost = 0.308
Epoch: 0002 Avg. cost = 0.084
Epoch: 0003 Avg. cost = 0.058
Epoch: 0004 Avg. cost = 0.043
Epoch: 0005 Avg. cost = 0.034
Epoch: 0006 Avg. cost = 0.027
Epoch: 0007 Avg. cost = 0.020
Epoch: 0008 Avg. cost = 0.017
Epoch: 0009 Avg. cost = 0.015
Epoch: 0010 Avg. cost = 0.012


In [13]:
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy,
                           feed_dict={
                               X: mnist.test.images.reshape(-1, 28, 28, 1),
                               Y: mnist.test.labels}))

Accuracy 0.981


## Dropout

In [16]:
# Fill in the details
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, dropout_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                         dropout_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1),
          'Avg. cost = {:,.3f}'.format(total_cost / total_batch))

correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy,
                           feed_dict={
                               X: mnist.test.images.reshape(-1, 28, 28, 1),
                               Y: mnist.test.labels, 
                           dropout_prob: 1}))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch: 0001 Avg. cost = 0.345
Epoch: 0002 Avg. cost = 0.108
Epoch: 0003 Avg. cost = 0.080
Epoch: 0004 Avg. cost = 0.063
Epoch: 0005 Avg. cost = 0.049
Epoch: 0006 Avg. cost = 0.043
Epoch: 0007 Avg. cost = 0.037
Epoch: 0008 Avg. cost = 0.032
Epoch: 0009 Avg. cost = 0.028
Epoch: 0010 Avg. cost = 0.025
Accuracy 0.9897


## 모든 layer에 dropout 추가

In [17]:
# Fill in the details
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, dropout_prob)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, dropout_prob)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, dropout_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                         dropout_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1),
          'Avg. cost = {:,.3f}'.format(total_cost / total_batch))

correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy,
                           feed_dict={
                               X: mnist.test.images.reshape(-1, 28, 28, 1),
                               Y: mnist.test.labels, 
                           dropout_prob: 1}))

Epoch: 0001 Avg. cost = 0.322
Epoch: 0002 Avg. cost = 0.103
Epoch: 0003 Avg. cost = 0.075
Epoch: 0004 Avg. cost = 0.059
Epoch: 0005 Avg. cost = 0.050
Epoch: 0006 Avg. cost = 0.043
Epoch: 0007 Avg. cost = 0.038
Epoch: 0008 Avg. cost = 0.035
Epoch: 0009 Avg. cost = 0.031
Epoch: 0010 Avg. cost = 0.028
Accuracy 0.9908


## Additional Conv. Layer

In [21]:
# Fill in the details
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W4 = tf.Variable(tf.random_normal([4*4*128, 256], stddev=0.01))
L4 = tf.reshape(L3, [-1, 4*4*128])
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, dropout_prob)

W5 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L4, W5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                         dropout_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1),
          'Avg. cost = {:,.3f}'.format(total_cost / total_batch))

correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy,
                           feed_dict={
                               X: mnist.test.images.reshape(-1, 28, 28, 1),
                               Y: mnist.test.labels, 
                           dropout_prob: 1}))

Epoch: 0001 Avg. cost = 0.381
Epoch: 0002 Avg. cost = 0.107
Epoch: 0003 Avg. cost = 0.076
Epoch: 0004 Avg. cost = 0.058
Epoch: 0005 Avg. cost = 0.047
Epoch: 0006 Avg. cost = 0.038
Epoch: 0007 Avg. cost = 0.032
Epoch: 0008 Avg. cost = 0.029
Epoch: 0009 Avg. cost = 0.026
Epoch: 0010 Avg. cost = 0.021
Accuracy 0.9906


## High-Level API

In [23]:
# Fill in the details
# Fill in the details
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

L1 = tf.layers.conv2d(X, 32, [3,3], activation='relu', padding='same')
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])

L2 = tf.layers.conv2d(L1, 64, [3,3], activation='relu', padding='same')
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation='relu')
model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1),
          'Avg. cost = {:,.3f}'.format(total_cost / total_batch))

correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy,
                           feed_dict={
                               X: mnist.test.images.reshape(-1, 28, 28, 1),
                               Y: mnist.test.labels}))

Epoch: 0001 Avg. cost = 0.165
Epoch: 0002 Avg. cost = 0.046
Epoch: 0003 Avg. cost = 0.031
Epoch: 0004 Avg. cost = 0.024
Epoch: 0005 Avg. cost = 0.018
Epoch: 0006 Avg. cost = 0.013
Epoch: 0007 Avg. cost = 0.011
Epoch: 0008 Avg. cost = 0.010
Epoch: 0009 Avg. cost = 0.008
Epoch: 0010 Avg. cost = 0.006
Accuracy 0.9883


## Multi-GPU Training

In [ ]:
# Fill in the d
def model(X, reuse=True)
    with tf.variable.scope('L1', reuse=reuse):
        L1 = tf.layers.conv2d(X,32,[3,3], activation='relu', reuse=reuse)
        L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
        
    with tf.variable.scope('L2', reuse=reuse):
        L2 = tf.layers.conv2d(L1,32,[3,3], activation='relu', reuse=reuse)
        L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
        
    with tf.variable.scope('L3', reuse=reuse):
        L3 = tf.layers.conv2d(L2,32,[3,3], activation='relu', reuse=reuse)
        L3 = tf.layers.max_pooling2d(L3, [2,2], [2,2])
        
    with tf.variable.scope('L4', reuse=reuse):
        L4 = tf.layers.flatten(L3)
        L4 = tf.layers.dense(L4, 256, activation='relu', reuse=reuse)
        L4 = tf.layers.dropout(L4, dropout_prob)
    
    with tf.variable_scope('OUT', reuse=reuse):
        out = tf.layers.dense(L4, 10)
    
    return out

batch_size = 100
gpu_num = 4
batch_size *= gpu_num
total_batch = int(mnist.train.num_examples/batch_size)